In [1]:
import numpy as np
import pandas as pd
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense,LSTM
from keras.layers.embeddings import Embedding
top_words=1000

Using TensorFlow backend.


In [2]:
# got error when loading so gone for this: https://stackoverflow.com/questions/55890813/how-to-fix-object-arrays-cannot-be-loaded-when-allow-pickle-false-for-imdb-loa
# save np.load
np_load_old = np.load

# modify the default parameters of np.load
np.load = lambda *a,**k: np_load_old(*a, allow_pickle=True, **k)

#for this step got error did the above below steps from above link
(x_train,y_train),(x_test,y_test)=imdb.load_data(num_words=top_words)


np.load = np_load_old

In [3]:
x_train.shape

(25000,)

In [4]:
y_train.shape

(25000,)

In [5]:
x_test.shape

(25000,)

In [6]:
y_test.shape

(25000,)

In [7]:
print(len(x_train[0]),len(x_train[1]),len(x_train[2]))

218 189 141


In [8]:
y_train[0]

1

In [ ]:
# top_words not worked  do manually 
#took 1000 not got because indices[3,74] = 3078 is not in [0, 1000) eroor so taking 5000
x_train=x_train[:5000]
y_train=y_train[:5000]
x_test=x_test[:5000]
y_test=y_test[:5000]
top_words=5000



In [ ]:
#padding
length=220 #taking small as i got max word length is 218 above they are actually aranged in sorted order in dataset
from keras.preprocessing import sequence
x_train=sequence.pad_sequences(x_train,maxlen=length)
x_test=sequence.pad_sequences(x_test,maxlen=length)



In [ ]:

model = Sequential()
model.add(Embedding(top_words,32, input_length=length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 220, 32)           160000    
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               53200     
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 101       
Total params: 213,301
Trainable params: 213,301
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
results=model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

Train on 5000 samples, validate on 5000 samples
Epoch 1/5
5000/5000 [==============================] - 52s 10ms/step - loss: 0.6810 - acc: 0.5630 - val_loss: 0.6670 - val_acc: 0.6084
Epoch 2/5
5000/5000 [==============================] - 47s 9ms/step - loss: 0.6356 - acc: 0.6844 - val_loss: 0.6010 - val_acc: 0.6860
Epoch 3/5
5000/5000 [==============================] - 48s 10ms/step - loss: 0.5189 - acc: 0.7528 - val_loss: 0.5556 - val_acc: 0.7118
Epoch 4/5
 960/5000 [====>.........................] - ETA: 34s - loss: 0.4448 - acc: 0.7896

In [ ]:
import matplotlib.pyplot as plt
def plt_dynamic(x, y, y_1, ax, colors=['b']):
    ax.plot(x, y, 'b', label="Train Loss")
    ax.plot(x, y_1, 'r', label="Test Loss")
    if len(x)==1:
        plt.legend()
    fig.canvas.draw()

In [ ]:

# Final evaluation of the model
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

fig,ax = plt.subplots(1,1)
ax.set_xlabel('epoch') ; 
ax.set_ylabel('Categorical Crossentropy Loss')
x = list(range(1,6+1))

vy = results.history['val_loss']
ty = results.history['loss']
print(x_test.shape,len(vy),len(ty))
#plt_dynamic(x, vy, ty, ax) #got error

In [133]:
#increasing accuarcy by trainng it more times
results=model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=20, batch_size=64)

Train on 5000 samples, validate on 5000 samples
Epoch 1/20
5000/5000 [==============================] - 43s 9ms/step - loss: 0.3345 - acc: 0.8598 - val_loss: 0.5018 - val_acc: 0.8078
Epoch 2/20
5000/5000 [==============================] - 51s 10ms/step - loss: 0.3571 - acc: 0.8476 - val_loss: 0.4820 - val_acc: 0.7982
Epoch 3/20
5000/5000 [==============================] - 48s 10ms/step - loss: 0.3094 - acc: 0.8760 - val_loss: 0.4639 - val_acc: 0.7942
Epoch 4/20
5000/5000 [==============================] - 49s 10ms/step - loss: 0.3063 - acc: 0.8754 - val_loss: 0.4863 - val_acc: 0.8148
Epoch 5/20
5000/5000 [==============================] - 51s 10ms/step - loss: 0.3028 - acc: 0.8750 - val_loss: 0.4958 - val_acc: 0.7732
Epoch 6/20
5000/5000 [==============================] - 50s 10ms/step - loss: 0.2763 - acc: 0.8884 - val_loss: 0.4771 - val_acc: 0.8174
Epoch 7/20
5000/5000 [==============================] - 50s 10ms/step - loss: 0.2552 - acc: 0.8992 - val_loss: 0.4848 - val_acc: 0.7920
E

In [134]:
scores = model.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))



5000/5000 [==============================] - 14s 3ms/step
Accuracy: 80.20%


# using dropout

In [ ]:
from keras.layers import Dropout
model1 = Sequential()
model1.add(Embedding(top_words,32, input_length=length))
model1.add(Dropout(0.2))
model1.add(LSTM(100))
model1.add(Dropout(0.2))
model1.add(Dense(1, activation='sigmoid'))
model1.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model1.summary())


In [ ]:
results=model1.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

In [ ]:
scores = model1.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))


#Keras provides this capability with parameters on the LSTM layer, the dropout for configuring the input dropout and recurrent_dropout for configuring the recurrent dropout

In [ ]:
model2 = Sequential()
model2.add(Embedding(top_words, 32, input_length=length))
model2.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
model2.add(Dense(1, activation='sigmoid'))
model2.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model2.summary())



In [ ]:
results=model2.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

In [ ]:
scores = model2.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))

# cnn after embedding

In [ ]:
from keras.layers.convolutional import Conv1D,MaxPooling1D
model3 = Sequential()
model3.add(Embedding(top_words, 32, input_length=length))
model3.add(Conv1D(filters=32, kernel_size=3, padding='same', activation='relu'))
model3.add(MaxPooling1D(pool_size=2))
model3.add(LSTM(100))
model3.add(Dense(1, activation='sigmoid'))
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model3.summary())


In [ ]:
results=model3.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=5, batch_size=64)

In [ ]:
scores = model3.evaluate(x_test, y_test, verbose=1)
print("Accuracy: %.2f%%" % (scores[1]*100))